In [1]:
import openai
import pandas as pd
import numpy as np
import re
import json
import time
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
openai.api_key = ""

## Import our topics

In [3]:
#keep sorting as in keywords like in paper
keywords=pd.read_csv('../Data/Model/LDA_final_40_topics_summary_table.csv')
names=pd.read_csv('../Data/LDA_final_topic_names.csv')#.New_short.tolist()
names=names.iloc[keywords.Topic_Num.astype(int).tolist(), ]
str_order=[str(i+1) for i in keywords.Topic_Num.astype(int).tolist()]

In [4]:
numbered_topics = [f"{i+1:2}    {line}" for i, line in enumerate(names['New_short'])]
numbered_topics = '\n'.join(numbered_topics)

numbered_topics_long = [f"{i+1:2}    {line}" for i, line in enumerate(names['New_long'])]
numbered_topics_long = '\n'.join(numbered_topics_long)

numbered_keywords = [f"{i+1:2}    {line}" for i, line in enumerate(keywords['Keywords'])]
numbered_keywords = '\n'.join(numbered_keywords)

# Chat 

In [5]:
def get_gpt_response(messages):
    response=openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=1,
        n=2
    )
    return response['choices'][0]['message']['content']

def get_gpt_response_multiple(messages, N):
    response=openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=1,
        n=N
    )
    return response['choices']#[0]['message']['content']

In [6]:
#Get example answer
prompt="I will provide you with a list of keywords for topics. Please give the name for each topic and rate them from 1 to 5 based on how polarizing they are."

messages=[{"role": "user", "content": prompt}]
answer=get_gpt_response(messages)
print(answer)

Sure, I'll be glad to help you with that. Please provide me with the list of keywords, and I'll give you the topic names and rate them based on their polarizing nature.


## Topic list

In [7]:
prompt="""I will provide you with a list of topics. Please rate them from 1 to 5 based on how polarizing they are."""

answer="""Certainly! Please provide me with the list of topics,
and I'll rate them on a scale of 1 to 5, with 1 being least 
polarizing and 5 being highly polarizing.""".replace('\n',' ')

topics=numbered_topics_long

messages = [{"role": "user", "content": prompt},
            {"role": "assistant", "content": answer}, 
            {"role": "user", "content": topics}]
    

In [8]:
result=get_gpt_response(messages)
result_10=get_gpt_response_multiple(messages, 10)
print(result)

Here's the rating of the topics on a scale of 1 to 5, with 1 being least polarizing and 5 being highly polarizing:

1. Family - 1
2. Politics - 5
3. Emotions and Feelings - 2
4. Coronavirus Pandemic - 4
5. Books - 1
6. Architecture - 1
7. Money, Personal Finance - 2
8. New York City - 2
9. Music/Movies - 1
10. Health Research, Lifestyle Advice - 2
11. Nature - 1
12. Black Lives Matter - 4
13. Women's Issues, Sexual Harassment - 4
14. Donald Trump - 5
15. Elections - 5
16. Joe Biden - 4
17. Political Investigations - 4
18. Public Health and Medicine - 3
19. Racial Identity and History - 4
20. Business - 3
21. Social Media - 3
22. Education, School System - 3
23. Supreme Court and Judicial System - 4
24. Food - 2
25. World News - 3
26. American Military - 3
27. Russia - 4
28. Covid Vaccine - 4
29. China, India, International Travel - 3
30. Real Estate - 2
31. Power, Energy Supply, and Climate - 4
32. Sports - 1
33. Art, Planes - 1
34. Science - 2
35. Covid Protection - 3
36. Israel - 4
3

In [9]:
def extract_numbers(text):
    pattern = r'\b(\d+)\s*$'
    matches = re.findall(pattern, text, re.MULTILINE)
    numbers = [int(match) for match in matches]
    return numbers 

text = result
numbers = extract_numbers(text)

len(numbers)

40

In [10]:
#Save all as json:
# with open("../Data/Gpt/topic_names_score_081823.json", "w") as outfile:
#     #json.dump(result_10, outfile)

topic_names_score = pd.DataFrame({'Num': names.Topic_Num, 'Topic': names.New_short}).reset_index(drop=True)

for i in range(0,len(result_10)):
    numbers=extract_numbers(result_10[i]['message']['content'])
    if len(numbers)==40:
        topic_names_score["run"+str(i+1)]=numbers
    else:
        topic_names_score["run"+str(i+1)]=[np.nan] * 40       
topic_names_score["avg"]=topic_names_score.loc[:,"run1":"run10"].mean(axis=1)

topic_names_score.head()


,Num,Topic,run1,run2,run3,run4,run5,run6,run7,run8,run9,run10,avg
0,29,Family,1,1,1,1,2,1,1,1,1,1,1.1
1,22,Politics,5,5,5,5,5,5,5,5,5,5,5.0
2,7,Feelings,2,2,2,2,1,1,2,1,2,2,1.7
3,16,Coronavirus,4,5,5,4,5,4,5,4,4,4,4.4
4,12,Books,1,1,2,1,1,1,1,1,1,1,1.1


In [11]:
#topic_names_score.to_csv('../Data/topic_names_score_081823.csv', index=False)
#topic_names_score=pd.read_csv('../Data/topic_names_score_081823.csv')

## Keywords

In [12]:
prompt="""I will provide you with a list of keywords for topics. 
Please give the name for each topic and rate them from 1 to 5 based on how polarizing they are.""".replace('\n',' ')

answer="""Sure! Please provide me with the list of keywords for topics and 
I'll be happy to give you the names for each topic and rate their 
level of polarization from 1 to 5, with 1 being least polarizing 
and 5 being highly polarizing.""".replace('\n',' ')

keywords=numbered_keywords

messages = [{"role": "user", "content": prompt},
            {"role": "assistant", "content": answer}, 
            {"role": "user", "content": keywords}]


In [13]:
result=get_gpt_response(messages)
result_10=get_gpt_response_multiple(messages, 10)
print(result)

Here are the topics and their respective polarization ratings:

1. Family: 1
2. Political: 4
3. Emotions: 1
4. Virus: 4
5. Literature: 1
6. Architecture: 1
7. Economy: 4
8. City: 2
9. Entertainment: 1
10. Research: 2
11. Nature: 1
12. Policing: 4
13. Gender: 3
14. Trump: 4
15. Political Parties: 4
16. Biden: 4
17. Legal: 4
18. Healthcare: 4
19. Race: 4
20. Business: 3
21. Social Media: 2
22. Education: 2
23. Court: 4
24. Food: 1
25. Europe: 1
26. Military: 4
27. Russia: 4
28. Vaccination: 4
29. China: 4
30. Housing: 2
31. Climate: 4
32. Sports: 1
33. Art: 1
34. Science: 1
35. COVID-19: 2
36. Israel/Palestine: 4
37. Religion: 4
38. Horse Racing: 2
39. Animals: 1
40. Jewish Culture: 3

Please note that these ratings are subjective and based on general perceptions of the topics. Different individuals may perceive the topics differently.


In [14]:
#Save all as json:
# with open("../../Data/Gpt/topic_keywords_score_081823.json", "w") as outfile:
#     #json.dump(result_10, outfile)


#save numbers to df:
topic_keywords_score = pd.DataFrame({'Num': names.Topic_Num, 'Topic': names.New_short}).reset_index(drop=True)

for i in range(0,len(result_10)):
    numbers=extract_numbers(result_10[i]['message']['content'])
    if len(numbers)==40:
        topic_keywords_score["run"+str(i+1)]=numbers
    else:
        topic_keywords_score["run"+str(i+1)]=[np.nan] * 40
        
topic_keywords_score["avg"]=topic_keywords_score.loc[:,"run1":"run10"].mean(axis=1)

#topic_keywords_score.to_csv('../Data/topic_keywords_score_081823.csv', index=False)
topic_keywords_score.head()

,Num,Topic,run1,run2,run3,run4,run5,run6,run7,run8,run9,run10,avg
0,29,Family,1,1,2,1,1,2,NaN,2,NaN,1,1.375
1,22,Politics,5,4,5,4,5,4,NaN,4,NaN,5,4.500
2,7,Feelings,2,2,2,2,2,3,NaN,1,NaN,2,2.000
3,16,Coronavirus,4,4,4,4,4,5,NaN,4,NaN,4,4.125
4,12,Books,1,1,1,1,1,2,NaN,1,NaN,1,1.125


In [15]:
#topic_keywords_score.to_csv('../Data/topic_keywords_score_081823.csv', index=False)
#topic_keywords_score=pd.read_csv('../Data/topic_keywords_score_081823.csv')

In [16]:
def extract_fractions(text):
    pattern = r'\b(\d+(\.\d+)?)\s*$'
    matches = re.findall(pattern, text, re.MULTILINE)
    numbers = [float(match[0]) for match in matches]
    return numbers[0]


## Correlation

In [29]:
def stand(my_df, by_row=0):
    updated=(my_df-my_df.mean(axis=by_row))/my_df.std(axis=by_row)
    return updated
topic_names_score.iloc[:, 2:12]=stand(topic_names_score.iloc[:, 2:12])
topic_names_score.avg=topic_names_score.iloc[:, 2:12].mean(axis=1)

topic_keywords_score.iloc[:, 2:12]=stand(topic_keywords_score.iloc[:, 2:12])
topic_keywords_score.avg=topic_keywords_score.iloc[:, 2:12].mean(axis=1)


summary=pd.read_csv('../Data/summary_results.csv', index_col=0).iloc[:,[1]].rename(columns=
    {'mean_polarization': 'survey'})
summary=summary.loc[names.New_short.to_list(), :]
summary["gpt_topics_avg"]=topic_names_score.avg.to_list()
summary["gpt_keywords_avg"]=topic_keywords_score.avg.to_list()
#summary.to_csv('../../Data/Gpt/summary_for081823.csv', index=False)
#summary.to_csv('../../Data/Gpt/summary_for081823_names.csv')
summary.head()


,survey,gpt_topics_avg,gpt_keywords_avg
Family,-0.32,-1.397705,-1.222141
Politics,0.89,1.608333,1.158909
Feelings,-0.34,-0.942725,-0.743366
Coronavirus,0.45,1.145691,0.882587
Books,-0.82,-1.385157,-1.411963
